# Build Fundamentals

In [1]:
import wrds
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset

In [2]:
prc_stmt = """
SELECT
    prc.gvkey,
    prc.datadate AS date,
    cshoc,
    prccd
FROM ( SELECT DISTINCT
        gvkey,
        iid
    FROM
        comp_global_daily.g_funda
    WHERE
        exchg = ANY (ARRAY [104, 132, 151, 154, 171, 172, 192, 194, 201, 209, 228, 256, 257, 273, 286])
        AND curcd = 'EUR') AS fund
    JOIN comp_global_daily.g_sec_dprc AS prc ON fund.gvkey = prc.gvkey
        AND fund.iid = prc.iid
WHERE
    curcdd = 'EUR'
    AND cshtrd IS NOT NULL
    AND datadate >= '1999-01-01'
    AND prc.gvkey = '100022'
"""

In [3]:
fund_stmt = """
SELECT
    gvkey,
    datadate AS date,
    loc AS country,
    indfmt AS industry,
    sicdesc AS classification,
    act,
    at AS att,
    capx,
    ceq,
    che,
    chee, 
    dlc,
    dltis,
    dltr,
    dltt,
    dp,
    dv,
    dvc,
    dvt,
    dlcch,
    ebit,
    ebitda,
    gdwl,
    icapt,
    intan,
    invt,
    ivaeq,
    ivao,
    ivst,
    lct,
    lt AS ltt,
    mib,
    nicon,
    oancf,
    pi,
    ppegt,
    pstk,
    prstkc,
    revt,
    sstk,
    COALESCE(txp, txpfs) AS txp,
    xrd
FROM
    comp.g_funda AS fund
    JOIN comp.r_siccd AS sic ON fund.sich::VARCHAR = sic.siccd
WHERE
    exchg = ANY (ARRAY [104, 107, 132, 151, 154, 171, 192, 194, 201, 209, 256, 257, 273, 276, 286])
    AND curcd = 'EUR'
    AND gvkey = '100022'
ORDER BY
    gvkey, datadate;
"""

In [4]:
def query_wrds(sql_stmt):
    with wrds.Connection(wrds_username="ottoriess") as db:
        data = db.raw_sql(
            sql_stmt,
            date_cols=["date", "datadate"],
        )
    return data

In [5]:
def query_fundamental():
    fund = (
        query_wrds(fund_stmt)
        .fillna(0)
        .astype(
            {
                "gvkey": "object",
                "country": "category",
                "industry": "category",
                "classification": "category",
            }
        )
    )
    return fund

In [6]:
def query_price():
    prc = query_wrds(prc_stmt)
    return prc

In [7]:
fund = query_fundamental()

Enter your WRDS username [ottoh]:ottoriess
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [8]:
prc = query_price()

Enter your WRDS username [ottoh]:ottoriess
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [9]:
def fill_year(df):
    first_date = df["date"].iloc[0]
    last_date = df["date"].iloc[-1]
    date_index = pd.date_range(
        pd.to_datetime(first_date), pd.to_datetime(last_date) + DateOffset(years=1), name="date"
    )
    return (
        df.drop("gvkey", axis=1)
        .set_index("date")
        .sort_index()
        .reindex(date_index, method="ffill")
    )

In [10]:
def build_fundamental(df):
    oa = df.att - df.che
    ol = df.att - df.dlc - df.dltt - df.mib - df.pstk - df.ceq
    chact = df.act - df.act.shift(1)
    chchee = df.chee - df.chee.shift(1)
    chlct = df.lct - df.lct.shift(1)
    chdlc = df.dlc - df.dlc.shift(1)
    chtxp = df.txp - df.txp.shift(1)
    chchee = df.chee - df.chee.shift(1)
    avg_at = (df.att + df.att.shift(1)) / 2
    nca = df.att - df.act - df.ivaeq
    ncl = df.ltt - df.lct - df.dltt
    ncoa = nca - ncl
    coa = df.act - df.che
    col = df.lct - df.dlc
    wc = df.act - df.che - df.lct + df.dlc
    fna = df.ivst + df.ivao
    fnl = df.dltt + df.dlc + df.pstk
    nfna = fna - fnl
    df = df.assign(
        acc=((chact - chchee) - (chlct - chdlc - chtxp) - df.dp) / avg_at,
        chcoa=(coa - coa.shift(1)) / df.att.shift(1),
        chcol=(col - col.shift(1)) / df.att.shift(1),
        chnncwc=(wc - wc.shift(1)) / df.att.shift(1),
        chnncoa=(ncoa - ncoa.shift(1)) / df.att.shift(1),
        chncoa=(nca - nca.shift(1)) / df.att.shift(1),
        chncol=(ncl - ncl.shift(1)) / df.att.shift(1),
        chnfa=nfna - nfna.shift(1) / df.att.shift(1),
        chlti=(df.ivao - df.ivao.shift(1)) / df.att.shift(1),
        chce=(df.ceq - df.ceq.shift(1)) / df.att.shift(1),
        chfl=(df.dltt + df.dlc + df.pstk - (df.dltt + df.dlc + df.pstk).shift(1))
        / df.att.shift(1),
        grii=(df.invt - df.invt.shift(1)) / ((df.att + df.att.shift(1)) / 2),
        ich=(df.invt - df.invt.shift(1)) / df.att.shift(1),
        igr=(df.invt - df.invt.shift(1)) / df.invt.shift(1),
        nwcch=(wc - wc.shift(1)) / df.att.shift(1),
        ta=((ncoa + wc + nfna) - (ncoa + wc + nfna).shift(1)) / df.att.shift(1),
        agr=df.att / df.att.shift(1),
        chnoa=(((oa - ol) / df.att.shift(1)) - (((oa - ol) / df.att.shift(1)).shift(1)))
        / df.att.shift(1),
        chppeia=((df.ppegt - df.ppegt.shift(1)) + (df.invt - df.invt.shift(1)))
        / df.att.shift(1),
        cdi=np.log((df.dltt + df.dlc) / (df.dltt.shift(5) + df.dlc.shift(5))),
        griltnoa=(
            ((oa - ol) / df.att.shift(1))
            - (((oa - ol) / df.att.shift(1)).shift(1))
            - ((chact - chchee) - (chlct - chdlc - chtxp) - df.dp) / avg_at
        ),
        inv=(df.capx / df.revt)
        / (
            (
                (df.capx.shift(1) / df.revt.shift(1))
                + (df.capx.shift(2) / df.revt.shift(2))
                + (df.capx.shift(3) / df.revt.shift(3))
            )
            / 3
        ),
        ndf=(df.dltis - df.dltr + df.dlcch) / ((df.att + df.att.shift(1)) / 2),
        nef=(df.sstk - df.prstkc - df.dv) / ((df.att + df.att.shift(1)) / 2),
        noa=(oa - ol) / df.att.shift(1),
        noach=(ncoa - ncoa.shift(1)) / df.att,
        txfin=(df.sstk - df.dv - df.prstkc + df.dltis - df.dltr) / df.att,
        ir=(df.icapt - df.icapt.shift(1)) / (df.ebit * (df.nicon / df.pi)),
        nopat_g=(df.icapt - df.icapt.shift(1)) / df.icapt,
        rev_cagr_3=((df.revt / df.revt.shift(3)) ** (1 / 3)) - 1,
        ebitda_cagr_3=((df.ebitda / df.ebitda.shift(3)) ** (1 / 3)) - 1,
    )
    df.loc[:, "acc":] = df.loc[:, "acc":].fillna(0).replace(np.inf, 0)
    return df

In [11]:
fund_build = fund.groupby("gvkey").apply(build_fundamental).groupby("gvkey").apply(fill_year)

In [12]:
prc_build = prc.set_index(["gvkey", "date"])

In [13]:
df = fund_build.join(prc_build)

In [14]:
df.head(1000)

country industry                           classification  \
gvkey  date                                                                   
100022 1999-12-31     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2000-01-01     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2000-01-02     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2000-01-03     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2000-01-04     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
...                   ...      ...                                      ...   
       2002-09-21     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2002-09-22     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2002-09-23     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2002-09-24     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   
       2002-09-25     DEU     INDL  MOTOR VEHICLES AND PASSENGER CAR BODIES   

                       act      att    capx      ceq     che    chee      dlc  \
gvkey  date                                                                     
100022 1999-12-31  28736.0  37507.0  2125.0   3883.0  2939.0  2055.0   7379.0   
       2000-01-01  28736.0  37507.0  2125.0   3883.0  2939.0  2055.0   7379.0   
       2000-01-02  28736.0  37507.0  2125.0   3883.0  2939.0  2055.0   7379.0   
       2000-01-03  28736.0  37507.0  2125.0   3883.0  2939.0  2055.0   7379.0   
       2000-01-04  28736.0  37507.0  2125.0   3883.0  2939.0  2055.0   7379.0   
...                    ...      ...     ...      ...     ...     ...      ...   
       2002-09-21  32791.0  51259.0  8047.0  10719.0  3344.0  2437.0  11720.0   
       2002-09-22  32791.0  51259.0  8047.0  10719.0  3344.0  2437.0  11720.0   
       2002-09-23  32791.0  51259.0  8047.0  10719.0  3344.0  2437.0  11720.0   
       2002-09-24  32791.0  51259.0  8047.0  10719.0  3344.0  2437.0  11720.0   
       2002-09-25  32791.0  51259.0  8047.0  10719.0  3344.0  2437.0  11720.0   

                   ...       nef       noa     noach     txfin       ir  \
gvkey  date        ...                                                    
100022 1999-12-31  ...  0.000000  0.000000  0.000000 -0.005732  0.00000   
       2000-01-01  ...  0.000000  0.000000  0.000000 -0.005732  0.00000   
       2000-01-02  ...  0.000000  0.000000  0.000000 -0.005732  0.00000   
       2000-01-03  ...  0.000000  0.000000  0.000000 -0.005732  0.00000   
       2000-01-04  ...  0.000000  0.000000  0.000000 -0.005732  0.00000   
...                ...       ...       ...       ...       ...      ...   
       2002-09-21  ... -0.006565  0.922397  0.253263 -0.005580  6.89527   
       2002-09-22  ... -0.006565  0.922397  0.253263 -0.005580  6.89527   
       2002-09-23  ... -0.006565  0.922397  0.253263 -0.005580  6.89527   
       2002-09-24  ... -0.006565  0.922397  0.253263 -0.005580  6.89527   
       2002-09-25  ... -0.006565  0.922397  0.253263 -0.005580  6.89527   

                    nopat_g  rev_cagr_3  ebitda_cagr_3        cshoc  prccd  
gvkey  date                                                                 
100022 1999-12-31  0.000000         0.0            0.0          NaN    NaN  
       2000-01-01  0.000000         0.0            0.0          NaN    NaN  
       2000-01-02  0.000000         0.0            0.0          NaN    NaN  
       2000-01-03  0.000000         0.0            0.0  622227918.0  29.10  
       2000-01-04  0.000000         0.0            0.0  622227918.0  28.20  
...                     ...         ...            ...          ...    ...  
       2002-09-21  0.499049         0.0            0.0          NaN    NaN  
       2002-09-22  0.499049         0.0            0.0          NaN    NaN  
       2002-09-23  0.499049         0.0            0.0  622227918.0  33.75  
       2002-09-24  0.499049         0.0            0.0  622227918.0  31.86  
       2002-09-25  0.499049         

In [31]:
#### Dividend yield ratio
dyr = ((df.dvt*10**6) / df.cshoc) / df.prccd
dyr
# Recheck here if dvt should be taken - at least for this example, only cash paid div (dv) has figures

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.000000
        2000-01-04    0.000000
                        ...   
        2020-12-27         NaN
        2020-12-28    0.037206
        2020-12-29    0.037373
        2020-12-30    0.037855
        2020-12-31         NaN
Length: 7672, dtype: float64

In [32]:
#### Price to earnings
pe = (df.cshoc * df.prccd) / (df.nicon*10**6)
pe

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03         inf
        2000-01-04         inf
                        ...   
        2020-12-27         NaN
        2020-12-28    9.082453
        2020-12-29    9.041669
        2020-12-30    8.926732
        2020-12-31         NaN
Length: 7672, dtype: float64

In [33]:
#### Price to cash flow
pocf = (df.cshoc * df.prccd) / (df.oancf*10**6)
pocf

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03     3.416383
        2000-01-04     3.310722
                        ...    
        2020-12-27          NaN
        2020-12-28    12.081001
        2020-12-29    12.026753
        2020-12-30    11.873870
        2020-12-31          NaN
Length: 7672, dtype: float64

In [30]:
#### Price to sales
psales = (df.cshoc * df.prccd) / (df.revt*10**6)
psales

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.526331
        2000-01-04    0.510053
                        ...   
        2020-12-27         NaN
        2020-12-28    0.424533
        2020-12-29    0.422627
        2020-12-30    0.417255
        2020-12-31         NaN
Length: 7672, dtype: float64

In [29]:
#### PEG
peg = (df.prccd/((df.nicon*10**6)/df.cshoc)) / ((((df.nicon*10**6)/df.cshoc)/(((df.nicon*10**6)/df.cshoc).shift(1)))-1)
peg

gvkey   date      
100022  1999-12-31    NaN
        2000-01-01    NaN
        2000-01-02    NaN
        2000-01-03    NaN
        2000-01-04    NaN
                     ... 
        2020-12-27    NaN
        2020-12-28    NaN
        2020-12-29    inf
        2020-12-30    inf
        2020-12-31    NaN
Length: 7672, dtype: float64

In [25]:
#### M/B ratio
mb = (df.cshoc * df.prccd) / (df.ceq*10**6)
mb

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    4.663104
        2000-01-04    4.518884
                        ...   
        2020-12-27         NaN
        2020-12-28    0.745746
        2020-12-29    0.742397
        2020-12-30    0.732960
        2020-12-31         NaN
Length: 7672, dtype: float64

In [35]:
#### EV Sales
evsales = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.revt*10**6)
evsales

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.917471
        2000-01-04    0.901193
                        ...   
        2020-12-27         NaN
        2020-12-28    1.409161
        2020-12-29    1.407254
        2020-12-30    1.401882
        2020-12-31         NaN
Length: 7672, dtype: float64

In [36]:
#### EV EBITDA
evebitda = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.ebitda*10**6)
evebitda

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    10.616493
        2000-01-04    10.428129
                        ...    
        2020-12-27          NaN
        2020-12-28     9.556102
        2020-12-29     9.543175
        2020-12-30     9.506743
        2020-12-31          NaN
Length: 7672, dtype: float64

In [37]:
#### EV EBIT
evebit = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.ebit*10**6)
evebit

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    33.902076
        2000-01-04    33.300566
                        ...    
        2020-12-27          NaN
        2020-12-28    13.329275
        2020-12-29    13.311243
        2020-12-30    13.260426
        2020-12-31          NaN
Length: 7672, dtype: float64

In [38]:
#### EV EBITDA-Capex
evebitdacpx = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / ((df.ebitda-df.capx)*10**6)
evebitdacpx

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    37.220321
        2000-01-04    36.559938
                        ...    
        2020-12-27          NaN
        2020-12-28    17.347741
        2020-12-29    17.324273
        2020-12-30    17.258135
        2020-12-31          NaN
Length: 7672, dtype: float64

In [39]:
#### EV Operating CF
evocf = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / ((df.oancf)*10**6)
evocf

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03     5.955251
        2000-01-04     5.849590
                        ...    
        2020-12-27          NaN
        2020-12-28    40.100663
        2020-12-29    40.046414
        2020-12-30    39.893532
        2020-12-31          NaN
Length: 7672, dtype: float64

In [40]:
#### R&D to Market Capitalization
rdmarket = (df.xrd*10**6) / (df.cshoc * df.prccd)
rdmarket

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.000000
        2000-01-04    0.000000
                        ...   
        2020-12-27         NaN
        2020-12-28    0.134537
        2020-12-29    0.135144
        2020-12-30    0.136884
        2020-12-31         NaN
Length: 7672, dtype: float64

In [45]:
#### Degree of Operating Leverage
dopl = ((df.ebit/df.ebit.shift(1))-1) / ((df.revt/df.revt.shift(1))-1)
dopl

gvkey   date      
100022  1999-12-31   NaN
        2000-01-01   NaN
        2000-01-02   NaN
        2000-01-03   NaN
        2000-01-04   NaN
                      ..
        2020-12-27   NaN
        2020-12-28   NaN
        2020-12-29   NaN
        2020-12-30   NaN
        2020-12-31   NaN
Length: 7672, dtype: float64